In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databrickseteproject167.dfs.core.windows.net/orders")

In [0]:
df = df.drop("_rescued_data")
df.display()

In [0]:
df = df.withColumn("order_date", to_timestamp(col("order_date")))
df = df.withColumn("year", year(col("order_date")))
df.display()

In [0]:
df1 = df.withColumn("dense_rank", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

In [0]:
df1 = df1.withColumn("rank", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

In [0]:
df1 = df1.withColumn("row_number", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

### **CLASSES -- OOP**

In [0]:
class windows:
    def dense_rank(self, df):
        return df.withColumn("dense_rank", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    def rank(self, df):
        return df.withColumn("rank", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    def row_number(self, df):
        return df.withColumn("row_number", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
w = windows()
dfnew = w.dense_rank(df)
dfnew = w.rank(dfnew)
dfnew = w.row_number(dfnew)

In [0]:
dfnew.display()

In [0]:
dfnew.write.format("delta").mode("overwrite").save("abfss://silver@databrickseteproject167.dfs.core.windows.net/orders")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databrickseteproject167_catalog.silver.orders
USING DELTA
LOCATION "abfss://silver@databrickseteproject167.dfs.core.windows.net/orders"